In [1]:
import os
import re
import networkx as nx
import plotly.graph_objects as go
import plotly.express as px
import re

In [2]:
def modify_dpd_file_in_place(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()

    with open(filename, 'w') as file:
        for line in lines:
            match = re.search(r'path="([^"]+)"', line)
            if match:
                path_value = match.group(1)
                file_name = path_value.split('.')[0]  # Extract file name before the first dot
                # Check if 'file=' already exists to avoid duplication
                if 'file=' not in line:
                    modified_line = line.rstrip()[:-2] + f', file="{file_name}", ];\n'
                    file.write(modified_line)
                else:
                    file.write(line)
            else:
                file.write(line)  # Write unchanged if no path found

def parse_dpd_file(file_path):
    """
    Parses a .dpd file and returns a dictionary of nodes and a list of edges.
    Each node entry includes its label and the attributes string.
    """
    nodes = {}
    edges = []
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line.startswith("N:"):
                # Expected format: N: id "label" [attributes, ...];
                parts = line.split(maxsplit=3)
                node_id = parts[1]
                label = parts[2].strip('"')
                attributes = parts[3] if len(parts) > 3 else ""
                nodes[node_id] = {"label": label, "attributes": attributes}
            elif line.startswith("E:"):
                parts = line.split()
                source = parts[1]
                target = parts[2]
                edges.append((source, target))
    return nodes, edges

def create_graph_from_dpd(file_path):
    """
    Creates a NetworkX directed graph from the .dpd file.
    """
    nodes, edges = parse_dpd_file(file_path)
    G = nx.DiGraph()
    for node_id, data in nodes.items():
        G.add_node(node_id, **data)
    G.add_edges_from(edges)
    return G

###########################################
# 2. Plotting Helper Functions (Graph Styles)
###########################################

def plot_graph_spring_layout(G, title="Dependency Graph (Spring Layout)"):
    """
    Plots the dependency graph using a spring layout (force-directed).
    """
    pos = nx.spring_layout(G, seed=42, k=3 / (len(G.nodes()) ** 0.5))
    # Prepare edge trace
    edge_x, edge_y = [], []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color="gray"),
        hoverinfo="none", mode="lines"
    )
    # Prepare node trace
    node_x, node_y, hover_text = [], [], []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        hover_text.append(
            f"ID: {node}<br>Label: {G.nodes[node]['label']}<br>"
            f"Attributes: {G.nodes[node].get('attributes', 'None')}"
        )
    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode="markers", hoverinfo="text", text=hover_text,
        marker=dict(size=10, color="blue", line=dict(width=2))
    )
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title=title,
                        titlefont_size=16,
                        showlegend=False,
                        hovermode="closest",
                        width=1600,
                        height=900,
                        margin=dict(b=0, l=0, r=0, t=40),
                        xaxis=dict(showgrid=False, zeroline=False),
                        yaxis=dict(showgrid=False, zeroline=False)
                    ))
    fig.show()

def plot_node_degree_histogram(file_path):
    """
    Computes the degree (in+out) for each node and plots a histogram.
    """
    G = create_graph_from_dpd(file_path)
    degrees = [deg for (_, deg) in G.degree()]
    fig = go.Figure(data=[go.Histogram(
        x=degrees,
        nbinsx=max(degrees) + 1,
        marker=dict(color='blue')
    )],
    layout=go.Layout(
        title="Node Degree Histogram",
        xaxis_title="Degree",
        yaxis_title="Count"
    ))
    fig.show()

def plot_graph_unique_file_colors(file_path):
    """
    Plots the dependency graph with nodes colored uniquely according to the file attribute.
    Extracts the file name from the node attributes using regex.
    """
    G = create_graph_from_dpd(file_path)
    pos = nx.spring_layout(G, seed=42, k=3 / (len(G.nodes()) ** 0.5))
    
    file_colors = {}
    color_palette = px.colors.qualitative.Plotly
    color_index = 0
    node_colors = []
    for node in G.nodes():
        attr = G.nodes[node].get("attributes", "")
        match = re.search(r'file="([^"]+)"', attr)
        file_name = match.group(1) if match else "Unknown"
        if file_name not in file_colors:
            file_colors[file_name] = color_palette[color_index % len(color_palette)]
            color_index += 1
        node_colors.append(file_colors[file_name])
    
    # Prepare edge trace
    edge_x, edge_y = [], []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color="gray"),
        hoverinfo="none", mode="lines"
    )
    # Prepare node trace
    node_x, node_y, hover_text = [], [], []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        attr = G.nodes[node].get("attributes", "None")
        hover_text.append(
            f"ID: {node}<br>Label: {G.nodes[node]['label']}<br>Attributes: {attr}"
        )
    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode="markers", hoverinfo="text", text=hover_text,
        marker=dict(size=10, color=node_colors, line=dict(width=2))
    )
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title="Dependency Graph (Colored by File)",
                        titlefont_size=16,
                        showlegend=False,
                        hovermode="closest",
                        width=1600,
                        height=900,
                        margin=dict(b=0, l=0, r=0, t=40),
                        xaxis=dict(showgrid=False, zeroline=False),
                        yaxis=dict(showgrid=False, zeroline=False)
                    ))
    fig.show()

def plot_graph_shaded_by_neighbors(file_path):
    """
    Plots the dependency graph with node colors shaded according to the number
    of unique neighbors (union of in- and out-neighbors).
    """
    G = create_graph_from_dpd(file_path)
    pos = nx.spring_layout(G, seed=42, k=3 / (len(G.nodes()) ** 0.5))
    
    # Compute neighbor count for each node.
    neighbor_count = {}
    for node in G.nodes():
        neighbors = set(G.predecessors(node)).union(set(G.successors(node)))
        neighbor_count[node] = len(neighbors)
    
    # Prepare edge trace
    edge_x, edge_y = [], []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color="gray"),
        hoverinfo="none", mode="lines"
    )
    # Prepare node trace using neighbor count as the color value.
    node_x, node_y, node_color, hover_text = [], [], [], []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        node_color.append(neighbor_count[node])
        hover_text.append(
            f"ID: {node}<br>Label: {G.nodes[node]['label']}<br>"
            f"Neighbors: {neighbor_count[node]}"
        )
    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode="markers", hoverinfo="text", text=hover_text,
        marker=dict(
            size=10,
            color=node_color,
            colorscale='Viridis',
            colorbar=dict(title='Neighbor Count'),
            line=dict(width=2)
        )
    )
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title="Dependency Graph (Shaded by Neighbor Count)",
                        titlefont_size=16,
                        showlegend=False,
                        hovermode="closest",
                        width=1600,
                        height=900,
                        margin=dict(b=0, l=0, r=0, t=40),
                        xaxis=dict(showgrid=False, zeroline=False),
                        yaxis=dict(showgrid=False, zeroline=False)
                    ))
    fig.show()

###########################################
# 3. Dependency Waves (Layered / Hierarchical Layout)
###########################################

def compute_dependency_waves(G):
    """
    Computes "waves" (or layers) for a dependency graph.
    Wave 1 consists of nodes with no incoming edges.
    Wave k contains nodes whose dependencies are all in waves < k.
    Returns a dictionary: {wave_number: [node_ids]}.
    """
    waves = {}
    remaining_nodes = set(G.nodes())
    current_wave = 1

    while remaining_nodes:
        # Nodes with no incoming edges (within the remaining nodes).
        wave_nodes = [n for n in remaining_nodes if all(pred not in remaining_nodes for pred in G.predecessors(n))]
        if not wave_nodes:
            raise ValueError("Cycle detected in dependency graph; cannot compute layers cleanly.")
        waves[current_wave] = wave_nodes
        remaining_nodes -= set(wave_nodes)
        current_wave += 1

    return waves

def assign_positions_by_wave(waves, horizontal_spacing=200, vertical_spacing=150):
    """
    Given a dictionary of waves, assign (x,y) positions for each node.
    Nodes in wave 1 are at the top (y=0), wave 2 are below, etc.
    Returns a dict mapping node -> (x, y).
    """
    pos = {}
    for wave, nodes in waves.items():
        num_nodes = len(nodes)
        for i, node in enumerate(nodes):
            x = i * horizontal_spacing - ((num_nodes - 1) * horizontal_spacing) / 2.0
            y = (wave - 1) * vertical_spacing
            pos[node] = (x, y)
    return pos

def plot_graph_layered_by_file(G, pos, title="Theorem Dependency Graph (Layered by Waves, Colored by File)"):
    """
    Plots the dependency graph using the provided positions (typically layered by dependency wave),
    with nodes colored uniquely according to the file attribute extracted from each node's attributes.
    Uses node size and figure dimensions matching the other plots.
    """
    # Build a mapping from file names to unique colors.
    file_colors = {}
    color_palette = px.colors.qualitative.Plotly  # Choose any qualitative palette.
    color_index = 0
    node_colors = []
    for node in G.nodes():
        attr = G.nodes[node].get("attributes", "")
        match = re.search(r'file="([^"]+)"', attr)
        file_name = match.group(1) if match else "Unknown"
        if file_name not in file_colors:
            file_colors[file_name] = color_palette[color_index % len(color_palette)]
            color_index += 1
        node_colors.append(file_colors[file_name])
    
    # Prepare edge trace
    edge_x, edge_y = [], []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=1, color="gray"),
        hoverinfo="none", mode="lines"
    )
    
    # Prepare node trace.
    node_x, node_y, hover_text = [], [], []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        hover_text.append(
            f"ID: {node}<br>Label: {G.nodes[node]['label']}<br>"
            f"Attributes: {G.nodes[node].get('attributes', 'None')}"
        )
    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode="markers", hoverinfo="text", text=hover_text,
        marker=dict(size=10, color=node_colors, line=dict(width=2))
    )
    
    # Create and show the figure with consistent dimensions.
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title=title,
                        titlefont_size=16,
                        showlegend=False,
                        hovermode="closest",
                        width=1600,
                        height=900,
                        margin=dict(b=0, l=0, r=0, t=40),
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
                    ))
    fig.show()

def visualize_dependency_waves(file_path):
    """
    Reads the .dpd file, computes dependency waves, assigns positions,
    and plots the layered dependency graph with nodes colored by file.
    Also demonstrates an alternative layout using Graphviz's 'dot' (if available).
    """
    G = create_graph_from_dpd(file_path)
    
    try:
        waves = compute_dependency_waves(G)
    except ValueError as e:
        print(e)
        return

    print("Computed Waves:")
    for wave, nodes in waves.items():
        print(f"Wave {wave}: {nodes}")

    # Option 1: Custom layered layout based on computed waves.
    pos = assign_positions_by_wave(waves, horizontal_spacing=200, vertical_spacing=150)
    plot_graph_layered_by_file(G, pos, title="Theorem Dependency Graph (Layered by Waves, Colored by File)")

    # Option 2: Use Graphviz's 'dot' layout for a hierarchical drawing (if available).
    try:
        pos_dot = nx.nx_agraph.graphviz_layout(G, prog="dot")
        plot_graph_layered_by_file(G, pos_dot, title="Theorem Dependency Graph (Graphviz Dot Layout, Colored by File)")
    except Exception as e:
        print("Graphviz layout not available:", e)

In [3]:
dpd_file_path = r"C:\Users\User\Documents\GitHub\autoformalization\src\temp\dgraph.dpd"

if os.path.exists(dpd_file_path):
    # Optional: Uncomment the next line to modify the .dpd file in place.
    # modify_dpd_file_in_place(dpd_file_path)
    
    # 1. Plot Dependency Graph using Spring Layout.
    G = create_graph_from_dpd(dpd_file_path)
    plot_graph_spring_layout(G, title="Dependency Graph (Spring Layout)")
    
    # 2. Plot Node Degree Histogram.
    plot_node_degree_histogram(dpd_file_path)
    
    # 3. Plot Dependency Graph with Unique Colors per File.
    plot_graph_unique_file_colors(dpd_file_path)
    
    # 4. Plot Dependency Graph Shaded by Neighbor Count.
    plot_graph_shaded_by_neighbors(dpd_file_path)
    
    # 5. Plot Layered Dependency Waves (colored by file).
    visualize_dependency_waves(dpd_file_path)
else:
    print(f"Error: {dpd_file_path} not found.")

Computed Waves:
Wave 1: ['186', '835', '947', '788', '841', '1290', '885', '1092', '1210', '1417', '57', '114', '647', '1130', '187', '1468', '299', '320', '1101', '1150', '955', '861', '1213', '1395', '1249', '1257', '445', '1264', '1463', '113', '355', '46', '25', '20', '53', '1172', '498', '103', '847', '367', '839', '79', '1128', '599', '956', '923', '748', '269', '244', '426', '871', '604', '918', '1288', '34', '149', '348', '786', '1226', '1472', '1057', '303', '368', '842', '1220', '159', '1155', '654', '913', '407', '988', '874', '1403', '1107', '979', '898', '933', '1296', '798', '516', '476', '85', '915', '261', '1370', '801', '350', '1129', '1418', '1106', '630', '672', '1123', '1243', '1444', '258', '1038', '436', '1237', '1025', '1309', '1382', '239', '791', '179', '732', '916', '639', '876', '764', '1440', '1301', '1097', '1269', '120', '1127', '201', '929', '110', '897', '1405', '474', '478', '886', '1300', '928', '403', '793', '997', '1420', '347', '614', '548', '642', 

Graphviz layout not available: requires pygraphviz http://pygraphviz.github.io/


In [4]:
# import re

# dpd_file_path = r"C:\Users\User\Documents\GitHub\autoformalization\src\temp\dgraph.dpd"

# def modify_dpd_file_in_place(filename):
#     with open(filename, 'r') as file:
#         lines = file.readlines()

#     with open(filename, 'w') as file:
#         for line in lines:
#             match = re.search(r'path="([^"]+)"', line)
#             if match:
#                 path_value = match.group(1)
#                 file_name = path_value.split('.')[0]  # Extract file name before the first dot
#                 # Check if 'file=' already exists to avoid duplication
#                 if 'file=' not in line:
#                     modified_line = line.rstrip()[:-2] + f', file="{file_name}", ];\n'
#                     file.write(modified_line)
#                 else:
#                     file.write(line)
#             else:
#                 file.write(line)  # Write unchanged if no path found

# # Example usage
# modify_dpd_file_in_place(dpd_file_path)


In [5]:
# import os
# import re
# import networkx as nx
# import plotly.graph_objects as go
# import plotly.express as px

# def parse_dpd_file(file_path):
#     """
#     Parses a .dpd file and returns a dictionary of nodes and a list of edges.
#     Each node entry includes its label and the attributes string.
#     """
#     nodes = {}
#     edges = []

#     with open(file_path, 'r') as file:
#         for line in file:
#             line = line.strip()
#             if line.startswith("N:"):
#                 # Split into at most 4 parts: "N:", id, label, and the rest (attributes)
#                 parts = line.split(maxsplit=3)
#                 node_id = parts[1]
#                 label = parts[2].strip('"')
#                 attributes = parts[3] if len(parts) > 3 else ""
#                 nodes[node_id] = {"label": label, "attributes": attributes}
#             elif line.startswith("E:"):
#                 parts = line.split()
#                 source = parts[1]
#                 target = parts[2]
#                 edges.append((source, target))
#     return nodes, edges

# def create_graph_from_dpd(file_path):
#     """
#     Creates a networkx DiGraph from the .dpd file.
#     """
#     nodes, edges = parse_dpd_file(file_path)
#     G = nx.DiGraph()
#     for node_id, data in nodes.items():
#         G.add_node(node_id, **data)
#     G.add_edges_from(edges)
#     return G

# def plot_dependency_graph(file_path):
#     """
#     Plots the dependency graph with blue nodes (the original plot).
#     """
#     G = create_graph_from_dpd(file_path)
#     pos = nx.spring_layout(G, seed=42, k=3 / (len(G.nodes) ** 0.5))
    
#     # Prepare edge trace
#     edge_x, edge_y = [], []
#     for edge in G.edges():
#         x0, y0 = pos[edge[0]]
#         x1, y1 = pos[edge[1]]
#         edge_x.extend([x0, x1, None])
#         edge_y.extend([y0, y1, None])
#     edge_trace = go.Scatter(
#         x=edge_x, y=edge_y,
#         line=dict(width=0.5, color="gray"),
#         hoverinfo="none", mode="lines"
#     )
    
#     # Prepare node trace
#     node_x, node_y, hover_text = [], [], []
#     for node in G.nodes():
#         x, y = pos[node]
#         node_x.append(x)
#         node_y.append(y)
#         hover_text.append(
#             f"ID: {node}<br>Label: {G.nodes[node]['label']}<br>"
#             f"Attributes: {G.nodes[node].get('attributes', 'None')}"
#         )
#     node_trace = go.Scatter(
#         x=node_x, y=node_y,
#         mode="markers", hoverinfo="text", text=hover_text,
#         marker=dict(size=10, color="blue", line=dict(width=2))
#     )
    
#     fig = go.Figure(data=[edge_trace, node_trace],
#                     layout=go.Layout(
#                         title="Dependency Graph",
#                         titlefont_size=16, showlegend=False, hovermode="closest",
#                         width=1600, height=900,
#                         margin=dict(b=0, l=0, r=0, t=40),
#                         xaxis=dict(showgrid=False, zeroline=False),
#                         yaxis=dict(showgrid=False, zeroline=False)
#                     ))
#     fig.show()

# def plot_node_degree_histogram(file_path):
#     """
#     Computes the degree (in+out) for each node in the graph and plots a histogram.
#     """
#     G = create_graph_from_dpd(file_path)
#     degrees = [deg for (_, deg) in G.degree()]
    
#     fig = go.Figure(data=[go.Histogram(
#         x=degrees,
#         nbinsx=max(degrees) + 1,  # one bin per degree value
#         marker=dict(color='blue')
#     )],
#     layout=go.Layout(
#         title="Node Degree Histogram",
#         xaxis_title="Degree",
#         yaxis_title="Count"
#     ))
#     fig.show()

# def plot_graph_unique_file_colors(file_path):
#     """
#     Plots the dependency graph with nodes colored according to the 'file' attribute.
#     This function extracts the file name from the node attributes using regex.
#     """
#     G = create_graph_from_dpd(file_path)
#     pos = nx.spring_layout(G, seed=42, k=3 / (len(G.nodes) ** 0.5))
    
#     # Build a mapping from file names to colors.
#     file_colors = {}
#     # Use one of Plotly's qualitative color sequences
#     color_palette = px.colors.qualitative.Plotly
#     color_index = 0
#     node_colors = []
#     for node in G.nodes():
#         attr = G.nodes[node].get("attributes", "")
#         # Look for file="SomeFile" in the attributes string.
#         match = re.search(r'file="([^"]+)"', attr)
#         file_name = match.group(1) if match else "Unknown"
#         if file_name not in file_colors:
#             file_colors[file_name] = color_palette[color_index % len(color_palette)]
#             color_index += 1
#         node_colors.append(file_colors[file_name])
    
#     # Prepare edge trace
#     edge_x, edge_y = [], []
#     for edge in G.edges():
#         x0, y0 = pos[edge[0]]
#         x1, y1 = pos[edge[1]]
#         edge_x.extend([x0, x1, None])
#         edge_y.extend([y0, y1, None])
#     edge_trace = go.Scatter(
#         x=edge_x, y=edge_y,
#         line=dict(width=0.5, color="gray"),
#         hoverinfo="none", mode="lines"
#     )
    
#     # Prepare node trace
#     node_x, node_y, hover_text = [], [], []
#     for node in G.nodes():
#         x, y = pos[node]
#         node_x.append(x)
#         node_y.append(y)
#         attr = G.nodes[node].get("attributes", "None")
#         hover_text.append(
#             f"ID: {node}<br>Label: {G.nodes[node]['label']}<br>Attributes: {attr}"
#         )
#     node_trace = go.Scatter(
#         x=node_x, y=node_y,
#         mode="markers", hoverinfo="text", text=hover_text,
#         marker=dict(size=10, color=node_colors, line=dict(width=2))
#     )
    
#     fig = go.Figure(data=[edge_trace, node_trace],
#                     layout=go.Layout(
#                         title="Dependency Graph (Colored by File)",
#                         titlefont_size=16, showlegend=False, hovermode="closest",
#                         width=1600, height=900,
#                         margin=dict(b=0, l=0, r=0, t=40),
#                         xaxis=dict(showgrid=False, zeroline=False),
#                         yaxis=dict(showgrid=False, zeroline=False)
#                     ))
#     fig.show()

# def plot_graph_shaded_by_neighbors(file_path):
#     """
#     Plots the dependency graph with node colors shaded according to the number
#     of unique neighbors (i.e. the union of in-neighbors and out-neighbors).
#     """
#     G = create_graph_from_dpd(file_path)
#     pos = nx.spring_layout(G, seed=42, k=3 / (len(G.nodes) ** 0.5))
    
#     # Compute the neighbor count for each node.
#     neighbor_count = {}
#     for node in G.nodes():
#         # For directed graphs, consider both predecessors and successors.
#         neighbors = set(G.predecessors(node)).union(set(G.successors(node)))
#         neighbor_count[node] = len(neighbors)
    
#     # Prepare edge trace
#     edge_x, edge_y = [], []
#     for edge in G.edges():
#         x0, y0 = pos[edge[0]]
#         x1, y1 = pos[edge[1]]
#         edge_x.extend([x0, x1, None])
#         edge_y.extend([y0, y1, None])
#     edge_trace = go.Scatter(
#         x=edge_x, y=edge_y,
#         line=dict(width=0.5, color="gray"),
#         hoverinfo="none", mode="lines"
#     )
    
#     # Prepare node trace with neighbor count as the color value.
#     node_x, node_y, node_color, hover_text = [], [], [], []
#     for node in G.nodes():
#         x, y = pos[node]
#         node_x.append(x)
#         node_y.append(y)
#         node_color.append(neighbor_count[node])
#         hover_text.append(
#             f"ID: {node}<br>Label: {G.nodes[node]['label']}<br>"
#             f"Neighbors: {neighbor_count[node]}"
#         )
#     node_trace = go.Scatter(
#         x=node_x, y=node_y,
#         mode="markers", hoverinfo="text", text=hover_text,
#         marker=dict(
#             size=10,
#             color=node_color,
#             colorscale='Viridis',  # change this to any colorscale you like
#             colorbar=dict(title='Neighbor Count'),
#             line=dict(width=2)
#         )
#     )
    
#     fig = go.Figure(data=[edge_trace, node_trace],
#                     layout=go.Layout(
#                         title="Dependency Graph (Shaded by Neighbor Count)",
#                         titlefont_size=16, showlegend=False, hovermode="closest",
#                         width=1600, height=900,
#                         margin=dict(b=0, l=0, r=0, t=40),
#                         xaxis=dict(showgrid=False, zeroline=False),
#                         yaxis=dict(showgrid=False, zeroline=False)
#                     ))
#     fig.show()

# # --- Example Usage ---

# dpd_file_path = r"C:\Users\User\Documents\GitHub\autoformalization\src\temp\dgraph.dpd"

# if os.path.exists(dpd_file_path):
#     # 1. Plot the original dependency graph.
#     plot_dependency_graph(dpd_file_path)
    
#     # 2. Plot the node degree histogram.
#     plot_node_degree_histogram(dpd_file_path)
    
#     # 3. Plot the dependency graph with unique node colors per file.
#     plot_graph_unique_file_colors(dpd_file_path)
    
#     # 4. Plot the dependency graph with node colors shaded by neighbor count.
#     plot_graph_shaded_by_neighbors(dpd_file_path)
# else:
#     print(f"Error: {dpd_file_path} not found.")
